In [1]:
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline

## Central Limit Theorem

In [ ]:
x=np.random.beta(1,5,size=(200000,))

In [ ]:
sns.distplot(x)

In [ ]:
k=[]

for i in range(10000):
    j=np.random.choice(x,1000,replace=False)
    k.append(np.mean(j))
    if i%1000==0:
        print(i)


In [ ]:
sns.distplot(k)

In [ ]:
m=np.random.uniform(size=20000)
t=np.random.uniform(size=20000)

In [ ]:
b=np.ones_like(t)-1.332*t

In [ ]:
b

In [ ]:
x=np.where(m>0.5,4+np.sqrt(b),4-np.sqrt(b))

In [ ]:
x=np.where(np.isnan(x),0,x)

In [ ]:
sns.distplot(x)

In [ ]:
k=[]

for i in range(10000):
    j=np.random.choice(x,1000,replace=False)
    k.append(np.mean(j))
    if i%1000==0:
        print(i)

In [ ]:
sns.distplot(k)

In [ ]:
x.mean(),np.array(k).mean()

In [ ]:
x.var()/1000,np.array(k).var()

## Single Sample T-Test

In [6]:
file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/winequality-white.csv'
wine_data=pd.read_csv(file,delimiter=';')

In [ ]:
wine_data

In [ ]:
from scipy.stats import ttest_1samp

In [ ]:
ttest_1samp(wine_data['fixed acidity'],popmean=6.82)

# how to do single tailed test or single sided tests

## Paired T-test

In [10]:
file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/hsb2.sas7bdat'
data=pd.read_sas(file)

In [11]:
data

,id,female,race,ses,schtyp,prog,read,write,math,science,socst
0,70.0,0.0,4.0,1.0,1.0,1.0,57.0,52.0,41.0,47.0,57.0
1,121.0,1.0,4.0,2.0,1.0,3.0,68.0,59.0,53.0,63.0,61.0
2,86.0,0.0,4.0,3.0,1.0,1.0,44.0,33.0,54.0,58.0,31.0
3,141.0,0.0,4.0,3.0,1.0,3.0,63.0,44.0,47.0,53.0,56.0
4,172.0,0.0,4.0,2.0,1.0,2.0,47.0,52.0,57.0,53.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...
195,31.0,1.0,2.0,2.0,2.0,1.0,55.0,59.0,52.0,42.0,56.0
196,145.0,1.0,4.0,2.0,1.0,3.0,42.0,46.0,38.0,36.0,46.0
197,187.0,1.0,4.0,2.0,2.0,1.0,57.0,41.0,57.0,55.0,52.0
198,118.0,1.0,4.0,2.0,1.0,1.0,55.0,62.0,58.0,58.0,61.0


In [ ]:
from scipy.stats import ttest_rel

In [ ]:
ttest_rel(data['read'],data['write'])

# Unpaired T-test

### Variance Test

In [ ]:
from scipy.stats import levene

In [ ]:
levene(data.loc[data['female']==1,'read'],data.loc[data['female']==0,'read'])

In [ ]:
# this means variances are equal

### now we go ahead with the test

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
ttest_ind(data.loc[data['female']==1,'read'],
          data.loc[data['female']==0,'read'],
          equal_var=True)

# Anova : for more than two groups

In [ ]:
wine_data['quality'].unique()

In [ ]:
alcohol_grps=[wine_data.loc[wine_data['quality']==q,'alcohol'].values 
              
              for q in wine_data['quality'].unique() ]

In [ ]:
wine_data['quality'].value_counts()

In [ ]:
from scipy.stats import f_oneway

In [ ]:
f_oneway(*alcohol_grps)


### Pairwise comparison between groups

In [7]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [8]:
tukey = pairwise_tukeyhsd(endog=wine_data['alcohol'],     # Data
                          groups=wine_data['quality'],   # Groups
                          alpha=0.05)          # Significance level

In [9]:
tukey.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
3,4,-0.1925,0.9,-0.9526,0.5675,False
3,5,-0.5362,0.3013,-1.2584,0.1861,False
3,6,0.2304,0.9,-0.4902,0.9509,False
3,7,1.0229,0.001,0.2975,1.7484,True
3,8,1.291,0.001,0.5338,2.0482,True
3,9,1.835,0.0132,0.231,3.439,True
4,5,-0.3436,0.0025,-0.6086,-0.0787,True
4,6,0.4229,0.001,0.1625,0.6833,True
4,7,1.2155,0.001,0.9419,1.489,True
4,8,1.4835,0.001,1.1343,1.8327,True


# Chisq Test for categorical data

### for a single column: test for expected proportions

In [12]:
from scipy.stats import chisquare

In [22]:
k=data['race'].value_counts()

In [23]:
k

4.0    145
1.0     24
3.0     20
2.0     11
Name: race, dtype: int64

In [24]:
chisquare(k.values,[72.5,12,10,5.5])
# let me come back to you on this

Power_divergenceResult(statistic=100.0, pvalue=1.5541594313896026e-21)

### For two columns

In [25]:
from scipy.stats import chi2_contingency

In [26]:
k=pd.crosstab(data['female'],data['ses']).values

In [27]:
k

array([[15, 47, 29],
       [32, 48, 29]])

In [28]:
res=chi2_contingency(k)

In [30]:
k=pd.crosstab(data['race'],data['ses']).values

In [33]:
chi2_contingency(k)[1]

0.0050642308532138445

# Linear Regression with p-values : using stats model

In [3]:
file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/loans data.csv'

ld=pd.read_csv(file)

In [4]:
ld.drop(['ID','Amount.Funded.By.Investors'],axis=1,
        inplace=True)

for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    ld[col]=ld[col].str.replace("%","")

for col in ['Amount.Requested', 'Interest.Rate','Debt.To.Income.Ratio',
            'Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    ld[col]=pd.to_numeric(ld[col],errors='coerce')
    

k=ld['FICO.Range'].str.split("-",expand=True)


k=k.astype(float)

ld['fico']=0.5*(k[0]+k[1])

del ld['FICO.Range']

# Notice that to apply string function on pandas data frame columns you need to str attribute
cat_cols=ld.select_dtypes(['object']).columns

cat_cols

for col in cat_cols:
    freqs=ld[col].value_counts()
    k=freqs.index[freqs>20][:-1]
    for cat in k:
        name=col+'_'+cat
        ld[name]=(ld[col]==cat).astype(int)
    del ld[col]
    print(col)
    

Loan.Length
Loan.Purpose
State
Home.Ownership
Employment.Length


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
ld_train,ld_test=train_test_split(ld,
                                  test_size=0.2,
                                  random_state=2)

In [7]:
for col in ld_train.columns:
    if ld_train[col].isnull().sum()>0:
        ld_train.loc[ld_train[col].isnull(),
                     col]=ld_train[col].mean()



for col in ld_test.columns:
    if ld_test[col].isnull().sum()>0:
        ld_test.loc[ld_test[col].isnull(),
                    col]=ld_train[col].mean()

/home/siddarth.jha@npci.org.in/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
ld_train.reset_index(drop=True,inplace=True)
x_train=ld_train.drop('Interest.Rate',axis=1)
y_train=ld_train['Interest.Rate']

In [10]:
import statsmodels.api as sm

In [12]:
x_train_new = sm.add_constant(x_train)

/home/siddarth.jha@npci.org.in/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [13]:
x_train_new.head()

,const,Amount.Requested,Debt.To.Income.Ratio,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,fico,Loan.Length_36 months,Loan.Purpose_debt_consolidation,...,Employment.Length_10+ years,Employment.Length_< 1 year,Employment.Length_2 years,Employment.Length_3 years,Employment.Length_5 years,Employment.Length_4 years,Employment.Length_1 year,Employment.Length_6 years,Employment.Length_7 years,Employment.Length_8 years
0,1.0,3250.0,21.44,5000.00,12.0,20564.0,0.0,677.0,1,1,...,0,0,0,1,0,0,0,0,0,0
1,1.0,16000.0,7.78,4166.67,6.0,4240.0,3.0,737.0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1.0,10000.0,1.69,8000.00,6.0,8974.0,0.0,797.0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,1.0,3600.0,10.00,5750.00,8.0,10766.0,2.0,672.0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1.0,14000.0,3.47,8750.00,6.0,9035.0,1.0,682.0,1,1,...,1,0,0,0,0,0,0,0,0,0


In [14]:
mod = sm.OLS( y_train,x_train_new)

In [15]:
res = mod.fit()

In [16]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          Interest.Rate   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     108.8
Date:                Fri, 06 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:14:37   Log-Likelihood:                -4270.2
No. Observations:                2000   AIC:                             8658.
Df Residuals:                    1941   BIC:                             8989.
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [18]:
vif = [variance_inflation_factor(x_train_new.values, i) for i in range(x_train_new.shape[1])]

In [19]:
list(zip(x_train_new.columns,vif))

[('const', 600.5977763595605),
 ('Amount.Requested', 1.6884941511241596),
 ('Debt.To.Income.Ratio', 1.3877036785078507),
 ('Monthly.Income', 1.477176690677716),
 ('Open.CREDIT.Lines', 1.3451066609912332),
 ('Revolving.CREDIT.Balance', 1.3842482919486563),
 ('Inquiries.in.the.Last.6.Months', 1.0779462802284037),
 ('fico', 1.181577159924854),
 ('Loan.Length_36 months', 1.301060458665705),
 ('Loan.Purpose_debt_consolidation', 11.343872438350404),
 ('Loan.Purpose_credit_card', 7.136744836363933),
 ('Loan.Purpose_other', 4.181198230947039),
 ('Loan.Purpose_home_improvement', 3.252537612634494),
 ('Loan.Purpose_major_purchase', 2.6530725310223215),
 ('Loan.Purpose_small_business', 2.5477592387144954),
 ('Loan.Purpose_car', 1.8631739796836584),
 ('Loan.Purpose_wedding', 1.6748579542072817),
 ('Loan.Purpose_medical', 1.5248921182158963),
 ('Loan.Purpose_moving', 1.485682132641911),
 ('State_CA', 3.1612260504341707),
 ('State_NY', 2.3879641914050618),
 ('State_TX', 1.943326310292955),
 ('State_